In [1]:
import h5py
import numpy as  np
from pathlib import Path

from source.preprocessHighFPS import yield_aggregate_bin_files, aggregate_bin_files, _to_analytic, _load_file, _chunk_to_fft, _get_radar_filenames


In [2]:
BASE_DIR = Path("D:/Studia/sium/semestr3/radar-project")  # change to your project dir
RADAR_DIR = BASE_DIR / "data/2023_03_21/chodzonko"  # day dir

In [3]:
data = yield_aggregate_bin_files('2023_03_21', 3,chunk_length=1, radar_dir=RADAR_DIR)


In [5]:
try:
    h5file = h5py.File('dataset.h5','r+')
except:
    print('File not exist, creating file')
    h5file = h5py.File('dataset.h5','w')

# jak będzimey zmieniać numerki to możemy w jednym datasecie mieć wszystkie pomiary
h5group =h5file.create_group('8') # numer pliku z a_list_of_samples
h5dset =  h5group.create_dataset(name='dataset_8',
                                 shape=(1, 3, 4, 512),
                                 maxshape=(None,3,4,512),
                                 dtype= np.complex128,
                                 compression='gzip', shuffle=True)


In [6]:
i = 0
while True:
    try:
        chunk = next(data)
        chunk = _to_analytic(chunk)
    except ValueError:
        print('File converted')
        print(f'Chunks number: {i}')
        break

    if i != 0:
        h5dset.resize(h5dset.shape[0]+1, axis=0)

    h5dset[i,:,:,:] = chunk[0,:,:,:]
    i+=1


File converted
Chunks number: 19884


In [7]:
h5file.close()

In [103]:
loaded_file = h5py.File('dataset.h5','r')
frames = loaded_file['13']['dataset_13']

In [104]:
frames[0]

array([[[  18.+138.j,  -68. +23.j,  -78.  +3.j, ...,  -45.  -8.j,
          -35. -31.j,   -2. -42.j],
        [-183.-123.j,  -50. +81.j,  -68. +56.j, ...,    5. -32.j,
           24.  -3.j,   22. +12.j],
        [-109. +64.j,  -87.+104.j,  -82. +89.j, ...,    9. -26.j,
           38. -17.j,   54. +22.j],
        [-171.+268.j,  186.+133.j,  189.+139.j, ...,  -64. +44.j,
          -43.  +4.j,  -15. -20.j]],

       [[ 116. +13.j,  -44. -57.j,  -31. -55.j, ...,  -29. -43.j,
           13. -51.j,   49. -34.j],
        [-182.+109.j,  134. +95.j,  141.+122.j, ...,  -41.  +5.j,
          -28. -28.j,   17. -16.j],
        [  59. +93.j,  -57. -14.j,  -45. -26.j, ...,  -16. -33.j,
           14. -49.j,   23. -31.j],
        [ -86.+139.j,  245.  +8.j,  269. +55.j, ...,  -43. -10.j,
          -32. -43.j,  -10. -34.j]],

       [[  95.+159.j,  -35.-101.j,   13.-114.j, ...,  -13. -71.j,
           27. -60.j,   81. -42.j],
        [-180. -95.j,   15. +78.j,    0. +84.j, ...,  -17.  -3.j,
          -1

In [105]:
loaded_file.close()